<a href="https://colab.research.google.com/github/quarentena/PathOfBuilding/blob/dev/extractionFromJira.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title RUN THIS TO PREP THE MACHINE

!pip install --upgrade orjson

In [ ]:
#@title RUN THIS TO DEFINE CLASSES AND METHODS

class _issue:
  open = None
  in_design = None
  ready_for_refinement = None
  in_refinement = None
  ready_for_development = None
  selected_for_development = None
  in_development = None
  in_review = None
  ready_for_testing = None
  in_testing = None
  ready_for_validation = None
  in_validation = None
  closed = None
  total_blocked_days = 0

  def __init__(self, filter_id, id, name, issue_type,status,due,fix_priority,labels,transitions,flagging,created,resolution,epic):
    self.filter_id = filter_id
    self.id = id
    self.name = name
    self.issue_type = issue_type
    self.status = status
    self.due = due
    self.fix_priority = fix_priority
    self.labels = labels
    self.transitions = transitions
    self.flagging = flagging
    self.created = created
    self.open = datetime.datetime.strptime(self.created, '%Y-%m-%dT%H:%M:%S.%f%z').astimezone(pytz.timezone("Brazil/East")).strftime("%Y-%m-%d")
    self.resolution = resolution
    self.epic = epic

  def __str__(self):
    return self.id + " - " + self.name + " - " + self.issue_type + " - " + self.status

  def getLabels(self):
    return ','.join(self.labels)

  def updateTransitions(self, mode):
    for otransition in self.transitions:
      match otransition.st_to:
        case "In Design":
          if (mode == 1 and self.in_design == None) or mode != 1:
            self.in_design = otransition.timestamp
        case "Ready for Refinement":
          if (mode == 1 and self.ready_for_refinement == None) or mode != 1:
            self.ready_for_refinement = otransition.timestamp
        case "In Refinement":
          if (mode == 1 and self.in_refinement == None) or mode != 1:
            self.in_refinement = otransition.timestamp
        case "Ready for Development" | "Verified":
          if (mode == 1 and self.ready_for_development == None) or mode != 1:
            self.ready_for_development = otransition.timestamp
        case "Selected for Development" | "To Plan":
          if (mode == 1 and self.selected_for_development == None) or mode != 1:
            self.selected_for_development = otransition.timestamp
        case "In Development" | "In Progress" | "Developing":
          if (mode == 1 and self.in_development == None) or mode != 1:
            self.in_development = otransition.timestamp
        case "In Review":
          if (mode == 1 and self.in_review == None) or mode != 1:
            self.in_review = otransition.timestamp
        case "Ready for Testing" | "Ready for QA" | "To Internal QA":
          if (mode == 1 and self.ready_for_testing == None) or mode != 1:
            self.ready_for_testing = otransition.timestamp
        case "In Testing" | "In QA" | "Internal QAing":
          if (mode == 1 and self.in_testing == None) or mode != 1:
            self.in_testing = otransition.timestamp
        case "Ready for Product Review" | "Awaiting PM Review" | "To QA":
          if (mode == 1 and self.ready_for_validation == None) or mode != 1:
            self.ready_for_validation = otransition.timestamp
        case "In Product Review" | "In PM Review" | "QAing":
          if (mode == 1 and self.in_validation == None) or mode != 1:
            self.in_validation = otransition.timestamp
        case "Closed" | "Done" | "Ready for Release":
          if (mode == 1 and self.closed == None) or mode != 1:
            self.closed = otransition.timestamp

  def calcBlockedTime(self):
    vtotalblock = 0
    vfinalblock = None
    vinitblock = None

    for oflagging in self.flagging:
      if self.selected_for_development is not None:
        if oflagging.flagged == 1:
          if oflagging.timestamp >= self.selected_for_development:
            vinitblock = datetime.datetime.strptime(oflagging.timestamp, '%Y-%m-%d')
          else:
            vinitblock = datetime.datetime.strptime(self.selected_for_development, '%Y-%m-%d')
        else:
          vfinalblock = datetime.datetime.strptime(oflagging.timestamp, '%Y-%m-%d')
          if vinitblock is None:
            vinitblock = vinitblock = datetime.datetime.strptime(self.selected_for_development, '%Y-%m-%d')

        if vfinalblock is not None and vinitblock is not None:
          vDelta = vfinalblock - vinitblock
          vtotalblock = vtotalblock + vDelta.days
          vfinalblock = None
          vinitblock = None
    if vfinalblock == None and vinitblock != None:
      current_day = datetime.datetime.strptime(datetime.datetime.now().astimezone(pytz.timezone("Brazil/East")).strftime("%Y-%m-%d"),'%Y-%m-%d')
      vDelta = current_day - vinitblock
      vtotalblock = vtotalblock + vDelta.days
    self.total_blocked_days = vtotalblock

  def updateEpic (self,all_epics):
    epic = next((epic for epic in all_epics if epic[0] == self.epic), None)
    if epic is not None:
      self.epic = epic[1]
    else:
      self.epic = "No Epic"

class _issue_transitions:
  def __init__(self,issue_id,st_from,st_to,timestamp):
    self.issue_id = issue_id
    self.st_from = st_from
    self.st_to = st_to
    self.timestamp = timestamp

  def __str__(self):
    return "From: " + self.st_from + " - To: " + self.st_to + " - When: " + self.timestamp

class _issue_flagging:
  def __init__ (self,issue_id,flagged,timestamp):
    self.issue_id = issue_id
    self.flagged = flagged
    self.timestamp = timestamp

  def __str__(self):
    if (self.flagged == 1):
      return "Flagged: True - When: " + self.timestamp
    else:
      return "Flagged: False - When: " + self.timestamp

class _jira:
  def __init__ (self,jira_login,jira_token,filter_id):
    self.jira_login = jira_login
    self.jira_token   = jira_token
    self.filter_id  = filter_id

  def getIssues(self):
    url = "https://liferay.atlassian.net/rest/api/latest/search"

    auth = HTTPBasicAuth(self.jira_login,
                        self.jira_token)

    headers = {
        "Accept": "application/json"
    }

    vstartAt = 0
    vTotal = 1
    all_issues = []
    epic_list = []

    while vstartAt <= vTotal:
      query = {
          'jql': 'filter=' + self.filter_id,
        #'jql': 'key=lps-181422',
          'fields': 'status,summary,key,reporter,changelog,issuetype,duedate,labels,created,resolution,customfield_10014',
          'expand': 'changelog',
          'startAt': vstartAt,
          'maxResults': 100
      }

      vstartAt = vstartAt + 100

      response = requests.request(
          "GET",
          url,
          headers=headers,
          auth=auth,
          params=query
      )

      print (response.status_code)
      jsonResult = orjson.loads(response.text)

      vTotal = jsonResult['total']

      vfix_priority = 2

      for issue in jsonResult['issues']:
        vresolution = ''
        vlabels =[]
        vtransitions= []
        vflagging =[]
        vid = issue['key']
        vname = issue['fields']['summary']
        vissue_type = issue['fields']['issuetype']['name']
        vstatus = issue['fields']['status']['name']
        vdue = issue['fields']['duedate']
        vcreated = issue['fields']['created']
        if issue['fields']['resolution'] != None:
          vresolution = issue['fields']['resolution']['name']
        vepic = issue['fields']['customfield_10014']
        vlabels = issue['fields']['labels']
        for history in issue['changelog']['histories']:
          for items in history['items']:
            if items['field'] == "status":
              transition = _issue_transitions(issue['key'],items['fromString'],items['toString'],datetime.datetime.strptime(history['created'], '%Y-%m-%dT%H:%M:%S.%f%z').astimezone(pytz.timezone("Brazil/East")).strftime("%Y-%m-%d"))
              vtransitions.append(transition)
            elif items['field'] == "Flagged" or items['field'] == "Blocked":
              if items['toString'] == "Impediment" or items['toString'] == "Yes":
                flagging = _issue_flagging(issue['key'],1,datetime.datetime.strptime(history['created'], '%Y-%m-%dT%H:%M:%S.%f%z').astimezone(pytz.timezone("Brazil/East")).strftime("%Y-%m-%d"))
              else:
                flagging = _issue_flagging(issue['key'],0,datetime.datetime.strptime(history['created'], '%Y-%m-%dT%H:%M:%S.%f%z').astimezone(pytz.timezone("Brazil/East")).strftime("%Y-%m-%d"))
              vflagging.append(flagging)
        oIssue = _issue(self.filter_id,vid,vname,vissue_type,vstatus,vdue,vfix_priority,vlabels,vtransitions,vflagging,vcreated,vresolution,vepic)

        all_issues.append(oIssue)
    return all_issues

  def getEpics(self,issues):

    url = "https://liferay.atlassian.net/rest/api/latest/search"

    auth = HTTPBasicAuth(self.jira_login,
                         self.jira_token)

    headers = {
        "Accept": "application/json"
    }

    vstartAt = 0
    vTotal = 1
    epic_list = []
    all_epics = []

    for issue in issues:
      if issue.epic not in epic_list and issue.epic is not None:
        epic_list.append(issue.epic)

    while vstartAt <= vTotal:

      query = {
          'jql': 'key in (' + ','.join(epic_list) + ')',
        #'jql': 'key=lps-181422',
          'fields': 'status,summary,key,reporter,changelog,issuetype,duedate,labels,created,resolution,customfield_12821,customfield_12822',
          'expand': 'changelog',
          'startAt': vstartAt,
          'maxResults': 100
      }

      vstartAt = vstartAt + 100

      response = requests.request(
          "GET",
          url,
          headers=headers,
          auth=auth,
          params=query
      )

      jsonResult = orjson.loads(response.text)

      vTotal = jsonResult['total']

      for epic in jsonResult['issues']:
        vid = epic['key']
        vsummary = epic['fields']['summary']
        vname = epic['fields']['summary']

        all_epics.append([vid,vname,vsummary])

    return all_epics


In [ ]:
#@title RUN THIS FOR EXTRACTING FROM JIRA
#Importing data from filter
import requests
from requests.auth import HTTPBasicAuth
import json
import pytz
import orjson, numpy
from getpass import getpass
import datetime

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

def main():
  logSize = 0
  filter_id = input("Filter Id:")
  jira_login = input("Jira login:")
  jira_token = 'your-token'
  sheet_url = input("Input GSheet URL:")

  jira = _jira(jira_login,jira_token,filter_id)

  all_issues = jira.getIssues()
  all_epics = jira.getEpics(all_issues)

  gc = gspread.authorize(creds)

  worksheet = gc.open_by_url(sheet_url).worksheet("rawData")

  worksheet.clear()

  header = ["Id","Link","Name","Open","In Design","Ready for Refinement","In Refinement","Ready for Development","Selected for Development","In Development","In Review","Ready for Testing","In Testing","Ready for Validation","In Validation","Done","Type","Status","Resolution","Blocked Days","Labels","Fix Priority","Epic"]

  worksheet.update('A1', [header])
  data = []

  for oIssue in all_issues:
    oIssue.updateTransitions(1) # 1 for first time transitioned to / 2 for last time it transitioned to
    oIssue.calcBlockedTime()
    oIssue.updateEpic(all_epics)
    vlabel = oIssue.getLabels()
    data.append([oIssue.id,"https://liferay.atlassian.net/jira/software/c/projects/LPS/issues/" + oIssue.id,oIssue.name,oIssue.open,oIssue.in_design,oIssue.ready_for_refinement,oIssue.in_refinement,oIssue.ready_for_development,oIssue.selected_for_development,oIssue.in_development,oIssue.in_review,oIssue.ready_for_testing,oIssue.in_testing,oIssue.ready_for_validation,oIssue.in_validation,oIssue.closed,oIssue.issue_type,oIssue.status,oIssue.resolution,oIssue.total_blocked_days,vlabel,oIssue.fix_priority,oIssue.epic])

  worksheet.update('A2', data, value_input_option='USER_ENTERED')

if __name__ == "__main__":
    main()

Filter Id:14630
Jira login:rafael.dantas@liferay.com
Input GSheet URL:https://docs.google.com/spreadsheets/d/1w52KPiXI0ZoQvUxIyN3oVCYhMZ02WwoGd-jESrDm87c/edit#gid=1712365984
400


KeyError: ignored